In [ ]:
# Code file written by Darshee for google colaboratory
# last editied 2/28 by Kat

In [ ]:
!pip install padasip

In [ ]:
import padasip as pa
import pandas as pd
import numpy as np
import os
import padasip as pa
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd "/content/drive/MyDrive/Colab Notebooks/OnCampus/LongCovid"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/OnCampus/LongCovid


In [ ]:
#Kat's Clean signal
f_path = "CovTest_filtered.xlsx"
test_bvp = pd.read_excel(f_path, header=0)
test_bvp.head()

,timestamp,BVP (64 Hz),BVP_filtered,EDA (4 Hz),TEMP (4 Hz),ACC X (32 Hz),ACC Y (32 Hz),ACC Z (32 Hz)
0,2023-09-27 12:48:00.016,8.89,0.090333,NaN,NaN,NaN,NaN,NaN
1,2023-09-27 12:48:00.031,9.13,0.137257,NaN,NaN,13.0,-6.0,62.0
2,2023-09-27 12:48:00.047,9.23,0.254039,NaN,NaN,NaN,NaN,NaN
3,2023-09-27 12:48:00.063,9.27,0.424330,NaN,NaN,13.0,-6.0,62.0
4,2023-09-27 12:48:00.078,9.35,0.643417,NaN,NaN,NaN,NaN,NaN


In [ ]:
directory = 'Data'

# iterate over files in that directory
for filename in os.listdir(directory):
    f_path = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f_path):
        final = pd.DataFrame()
        cov_bvp = pd.read_csv(f_path, header=0)
        blocks = [0,1,2,3,4,5,6]
        for block in blocks:
          bl = cov_bvp[cov_bvp['Block']== block]
          test_bvp0 = test_bvp.iloc[:len(bl)]
          n = 10
          dd = bl['BVP_filtered'].values #noisy signal
          xx = test_bvp0['BVP_filtered'].values #clean signal
          x = pa.input_from_history(xx, n)
          f = pa.filters.FilterNLMS(n=n, mu=0.04, w="random")
          y, e, w = f.run(dd[:len(x)], x)
          df = pd.DataFrame(y, columns = ['padasip_BVP_signal'])
          bl = bl.iloc[:len(df)]
          bl['padasip'] = df.iloc[:,0].values
          final = pd.concat([final, bl], ignore_index=True)
        final.to_csv(filename+ "_cleaned.csv")

In [ ]:
# just do one file at a time for the additional participants
f_path = "Cov23_lab_filtered.csv"


final = pd.DataFrame()
cov_bvp = pd.read_csv(f_path, header=0)
blocks = [0,1,2,3,4,5,6]
for block in blocks:
  bl = cov_bvp[cov_bvp['Block']== block]
  test_bvp0 = test_bvp.iloc[:len(bl)]
  n = 10
  dd = bl['BVP_filtered'].values #noisy signal
  xx = test_bvp0['BVP_filtered'].values #clean signal
  x = pa.input_from_history(xx, n)
  f = pa.filters.FilterNLMS(n=n, mu=0.04, w="random")
  y, e, w = f.run(dd[:len(x)], x)
  df = pd.DataFrame(y, columns = ['padasip_BVP_signal'])
  bl = bl.iloc[:len(df)]
  bl['padasip'] = df.iloc[:,0].values
  final = pd.concat([final, bl], ignore_index=True)
final.to_csv("Cov23_lab_filtered_cleaned.csv")